In [1]:
from auth import get_creds
from model import GoogleDrive, MimeType, get_local_df
from sync import load_status, save_status
from pathlib import Path
from utils import get_md5
import pandas as pd

In [2]:
creds, _ = get_creds('royce.melborn2@gmail.com')
drive = self = GoogleDrive(creds)
# file_list = drive.list_all_files()
# root = DriveFile.cloud(drive.get_root())
# root.path = Path('/home/royce/Downloads/drive')

# drive.generate_tree(root)
import logging
log = logging.getLogger('temp')

In [137]:
root_id = drive.get_root()['id']
root_path = Path('/home/royce/rdrive')

cloud_df = self.get_cloud_df(root_id, root_path)
local_df = get_local_df(root_path)

In [138]:
df = pd.merge(
    cloud_df.reset_index(),
    local_df,
    how="outer",
    indicator=True,
    on="local_path",
).set_index("local_path")

cloud_extra = df["_merge"] == "left_only"
to_download = cloud_extra & (df["cloud_mtime"] > status["start_time"])
to_delete_cloud = cloud_extra & (df["cloud_mtime"] <= status["start_time"])
local_extra = df["_merge"] == "right_only"
df.loc[local_extra, "local_name"] = df.loc[local_extra].apply(
    lambda x: x.name.name, axis=1
)
df.loc[local_extra, "cloud_name"] = df.loc[local_extra].apply(
    lambda x: x.name.stem
    if x["local_type"] in GoogleDrive.google_mimes
    else x.name.name,
    axis=1,
)
to_upload = local_extra & (df["local_mtime"] > status["end_time"])
to_delete_local = local_extra & (df["local_mtime"] <= status["end_time"])

to_update = (
    (df["_merge"] == "both")
    & (df["local_md5"] != df["cloud_md5"])
    & (~df["cloud_type"].isin(GoogleDrive.google_mimes))
)
to_update_local = df[
    to_update
    & (df["cloud_mtime"] > df["local_mtime"])
    & (df["local_type"] != "folder")
]
to_update_cloud = df[
    to_update
    & (df["cloud_mtime"] < df["local_mtime"])
    & (df["local_type"] != "folder")
]


In [139]:
df[to_download]

,id,cloud_name,mime_type,parent,url,cloud_mtime,cloud_md5,cloud_type,local_name,local_type,local_mtime,local_md5,_merge
local_path,,,,,,,,,,,,,


In [136]:
df[to_update_local]

ValueError: fill value must be in categories

In [4]:
path = Path('/home/royce/rdrive/Armory Backup(copy).pdf')

In [10]:
from datetime import datetime

stat = path.stat()
out = {}
for x in ['st_atime', 'st_mtime', 'st_ctime']:
    out[x] = str(datetime.fromtimestamp(getattr(stat, x)))

out

{'st_atime': '2020-03-21 22:03:28',
 'st_mtime': '2020-03-20 18:02:02',
 'st_ctime': '2020-03-22 02:57:34.377516'}